### Cardiac MRI View Prediction

The following notebook uses a trained neural network to predict the MRI view for each series in a directory of dicom files. 

First, import the necessary packages to run the analysis. The required libraries are pydicom, pandas, and tensorflow. The original analysis was run using python 3.6.8, pydicom 1.2.2, and tensorflow 2.4.1.

In [1]:
import os
import sys
import pydicom # pydicom is using the gdcm package for decompression
from multiprocessing import Pool
import numpy as np
import time 
import pandas as pd
from tqdm import tqdm
from itertools import repeat
import tensorflow as tf

print('Python: {}'.format(sys.version))
print('Pydicom: {}'.format(pydicom.__version__))
print('TensorFlow: {}'.format(tf.__version__))

Python: 3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]
Pydicom: 2.2.1
TensorFlow: 2.4.1


Tensorflow can be run with or without GPU support. If GPU support through tensorflow is enabled and available, the following code will display and available GPU.

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


If no GPU is found, or if you simply want to run the analysis using only CPU, we can configure tensorflow to only use CPU using the following code:

In [ ]:
# Only use CPU for right now - Set CPU as only available physical device
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

The analysis can be run over a single directory of dicom images (for one or more patients) or over a directory containing multiple subdirectories. Due to memory constraints, skip ahead to section 2.0 if your directory contains multiple patients. The code in section 2.0 runs the analysis iteratively over each subdirectory. Alternatively, if you only have one patient or a small dataset, you can run the complete analysis by running all of the code in section 1.0. 

The suggested directory structures for each approach are shown below:

#### Approach 1 (Section 1.0)

Run the analysis over all files in a directory and subdirectories (may run into memory issues if dataset is large).

```bash
├── DATA
    └── Patient1
        ├── 1.dcm
        ├── 2.dcm 
        ├── 3.dcm          
        └── ...            
```

#### Approach 2 (Section 2.0)

Run the analysis over each patient in a directory individually (i.e., analysis for patient 1, then analysis for patient 2.)

```bash
├── DATA
    ├── Patient1
    │   ├── 1.dcm
    │   ├── 2.dcm 
    │   ├── 3.dcm          
    │   └── ...   
    └── Patient2
        ├── 1.dcm
        ├── 2.dcm 
        ├── 3.dcm          
        └── ...          
```

### Section 1.0 - Small Dataset and/or Single Patient

Use the following code to run the analysis over all the files in a single directory. If you wish to run the analysis over each patient in a directory (recommended due to memory constraints), skip ahead to section 2.0. 

#### Parameters

Define the desired parameters for the code in the cell below. 

In [4]:
# PARAMETERS for the analysis
src = "../data/raw/dicoms/CHD10553/"              # PATH to the directory containing the desired DICOM files (str)
dst = "../data/processed/sorted/"                 # PATH to the output directory to save dicom files (only valid if save_dicoms = True) (str)

modelname = 'ResNet50'                            # The neural network to load and used (Options: VGG19, ResNet50, or Xception)
modelpath = '../models/'                          # PATH to the saved models (str)

use_multiprocessing = True                        # Use multiprocessing to read header info (True or False)

# parameters for postprocessing/saving
csv_path = '../reports/series_predictions.csv'    # PATH to save the generated csv file (only valide if create_csv = True) (str)
create_csv = True                                 # Save a .csv file with the series level view predictions (True or False)
save_files = True                                 # Save dicom files to new directory (dst) (True or False)
save_only_desired = True                          # Save only dicom files corresponding to desired views (True or False)
confidence_value = 0.9                            # Only save series if the confidence is > a certain value (set to 0 to save all desired series, regardless of confidence) (float 0-1.0)

The following cells are used to define the possible MRI view classes (n=7) and the views that are desired for cardiac modeling.

In [5]:
# define possible class predictions
classLabels = ['SA', '4CH', '2CH RT', 'RVOT', 'OTHER', '2CH LT', 'LVOT']
classes = sorted(classLabels, key = str)
print(classes)

['2CH LT', '2CH RT', '4CH', 'LVOT', 'OTHER', 'RVOT', 'SA']


In [6]:
# define the series that are needed/desired for cardiac modeling
desired_series = ['4CH', 'SA', '2CH RT', '2CH LT', 'LVOT', 'RVOT']

#### Import and Load Model

The following code loads the saved neural network that will be used for view prediction:

In [7]:
# load the appropriate model
if modelname == 'ResNet50':
    MODELPATH = os.path.join(modelpath, 'Resnet/082621_resnet.hdf5')
    model = tf.keras.models.load_model(MODELPATH)
    print(model.summary())

elif modelname == 'VGG19':
    MODELPATH = os.path.join(modelpath, 'VGG19/vgg19.hdf5')
    model = tf.keras.models.load_model(MODELPATH)
    print(model.summary())
    
elif modelname == 'Xception':
    MODELPATH = os.path.join(modelpath, 'XCEPTION/xception.hdf5')
    model = tf.keras.models.load_model(MODELPATH)
    print(model.summary())

else:
    print('Uknown model specified in parameters!')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf.cast (TFOpLambda)            (None, 224, 224, 3)  0           inputs[0][0]                     
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 224, 224, 3)  0           tf.cast[0][0]                    
__________________________________________________________________________________________________
tf.nn.bias_add (TFOpLambda)     (None, 224, 224, 3)  0           tf.__operators__.getitem[0][0]   
______________________________________________________________________________________________

#### Define Necessary Functions

Before we get started, we define a few useful functions that we will use throughout the analysis. 

In [8]:
def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    
    return string.lower()  

def preprocess(img):
    # format image into tensor, standardized to 0-255
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(tf.expand_dims(img, 2), (224,224))
    img = tf.image.grayscale_to_rgb(img)
    
    # standardize
    img = img / np.max(img)
    img = img * 255.
    
    return img

def predict_view(img, model=model):
    # make prediction on a single image
    pred = model.predict(tf.expand_dims(img, axis=0))
    pred = tf.argmax(pred, axis=-1)
    pred_view = classes[int(pred)]
    
    return pred_view

def batch_predict(batch, model=model):
    # make prediction on a batch of images
    pred = model.predict(batch)
    pred = tf.argmax(pred, axis=-1)
    pred_view = [classes[int(x)] for x in pred]
    
    return pred_view

def get_dicom_header(dicom_loc):
    # read dicom file and return header information and image
    ds = pydicom.read_file(dicom_loc, force=True)
   
    # get patient, study, and series information
    patientID = clean_text(ds.get("PatientID", "NA"))
    studyDescription = clean_text(ds.get("StudyDescription", "NA"))
    seriesDescription = clean_text(ds.get("SeriesDescription", "NA"))
   
    # generate new, standardized file name
    modality = ds.get("Modality","NA")
    studyInstanceUID = ds.get("StudyInstanceUID","NA")
    seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
    seriesNumber = ds.get('SeriesNumber', 'NA')
    instanceNumber = str(ds.get("InstanceNumber","0"))
       
    # load image data
    array = ds.pixel_array
    
    return patientID, dicom_loc, modality, seriesInstanceUID, seriesNumber, instanceNumber, array, seriesDescription

#### Read DICOM Headers

Before the neural network can be used to predict the MRI view, we load the necessary information from the dicom headers, such as patient IDs, series, study, modality, instance, and the corresponding raw images. 

In [12]:
print('Reading file list...')
unsortedList = []
for root, dirs, files in os.walk(src):
    for file in files: 
        if ".dcm" in file: # exclude non-dicoms, good for messy folders
            unsortedList.append(os.path.join(root, file))

print('%s files found.' % len(unsortedList))

if use_multiprocessing:
    with Pool(os.cpu_count()) as p:
        output = p.map(get_dicom_header, [dicom_loc for dicom_loc in unsortedList])
        
    print('Done!')

else:
    output = []
    for dicom_loc in tqdm(unsortedList):
        output.append(get_dicom_header(dicom_loc))


Reading file list...
2283 files found.
Done!


In [11]:
# generated pandas dataframe to store information from headers
df = pd.DataFrame(sorted(output), columns = ['Patient ID', 
                                             'Filename',
                                             'Modality',
                                             'Series ID', 
                                             'Series Number', 
                                             'Instance Number', 
                                             'Img', 
                                             'Series Description'])
df.head().transpose()

,0,1,2,3,4
Patient ID,chd1055302,chd1055302,chd1055302,chd1055302,chd1055302
Filename,../data/raw/dicoms/CHD10553/CHD1055302/CAP_CHD...,../data/raw/dicoms/CHD10553/CHD1055302/CAP_CHD...,../data/raw/dicoms/CHD10553/CHD1055302/CAP_CHD...,../data/raw/dicoms/CHD10553/CHD1055302/CAP_CHD...,../data/raw/dicoms/CHD10553/CHD1055302/CAP_CHD...
Modality,MR,MR,MR,MR,MR
Series ID,2.16.124.113543.6006.99.4513285645466080530,2.16.124.113543.6006.99.4513285645466080530,2.16.124.113543.6006.99.4513285645466080530,2.16.124.113543.6006.99.4513285645466080530,2.16.124.113543.6006.99.4575318287899584458
Series Number,1401,1401,1401,1401,701
Instance Number,42,23,54,33,16
Img,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
Series Description,4ch_sense,4ch_sense,4ch_sense,4ch_sense,4ch_sense


#### Make Predictions for Each Series

Now that we have the header info and images, we can make predictions for each series. The following code iterates over each series and makes predictions in batches. 

The code generates a confidence level, which ranges from 0-1.0. This value is calculated for each series by dividing the count of the most frequent prediction by the total number of predictions. For example, if a 30 frame series has 29 correct predictions of '4CH', but one incorrect prediction of 'OTHER', the confidence would be 0.97. 

In [13]:
output_series = []
# make predictions and calculate confidence values
for series in tqdm(set(df['Series ID'])):
    new = df[df['Series ID'] == series]
    
    dataset = tf.data.Dataset.from_tensor_slices([preprocess(x) for x in new['Img'].values])
    dataset = (dataset
             .batch(16)
             .prefetch(tf.data.experimental.AUTOTUNE))
    
    # record info for this series
    patient_id = new['Patient ID'].iloc[0]
    series_num = new['Series Number'].iloc[0]
    series_desc = new['Series Description'].iloc[0]
    frames = len(new)
    
    # make predictions over images
    views = batch_predict(dataset, model)

    # find unique predictions and confidence for that series
    u, count = np.unique(views, return_counts=True)
    count_sort_ind = np.argsort(-count)
    pred = u[count_sort_ind][0]
    conf = np.round(np.max(count) / np.sum(count), 2)
        
    output_series.append([patient_id.upper(), series, series_num, frames, series_desc, pred, conf])
    
output_series_df = pd.DataFrame(output_series, columns=['Patient ID', 'Series ID', 'Series Number', 'Frames', 'Series Description', 'Predicted View', 'Confidence'])
output_series_df

100%|██████████| 22/22 [00:32<00:00,  1.47s/it]


,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence
0,CHD1055302,2.16.124.113543.6006.99.4500190260729165514,1601,90,flow_bh_pa_sense,OTHER,1.00
1,CHD1055302,2.16.124.113543.6006.99.4544301574540994342,1201,60,lvot_sense,LVOT,1.00
2,CHD1055302,2.16.124.113543.6006.99.4569864505174670623,801,180,rvot_sense,RVOT,1.00
3,CHD1055302,2.16.124.113543.6006.99.7470716397106359430,2101,90,flow_bh_rpa_sense,OTHER,1.00
4,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.00
5,CHD1055302,2.16.124.113543.6006.99.7481029872254558644,2301,90,flow_bh_lpa_sense,OTHER,1.00
6,CHD1055302,2.16.124.113543.6006.99.7464229721673281694,2001,90,flow_bh_lpa_sense,OTHER,1.00
7,CHD1055302,2.16.124.113543.6006.99.4485983478598388729,1001,360,ax_sense,OTHER,1.00
8,CHD1055302,2.16.124.113543.6006.99.7423782743545741482,1901,90,flow_bh_pa_sense,OTHER,1.00
9,CHD1055302,2.16.124.113543.6006.99.4513285645466080530,1401,80,4ch_sense,4CH,1.00


#### Save Predictions

Now that the predictions have been made, we can save the pandas dataframe containing the predicted view for each series to a .csv file and the dicom files of the desired views for cardiac modeling to a new directory, depending on the parameters that were specified above. 

In [14]:
if create_csv:
    print('Saving .csv file with series predictions and info')
    
    if os.path.exists(csv_path):
        output_series_df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        output_series_df.to_csv(csv_path, mode='a', index=False)
        
    print('Done!')

if save_files:
    print('Saving dicom files to new folder...')
    for series in tqdm(output_series_df['Series ID']):
        new = df[df['Series ID'] == series]
        series_df = output_series_df[output_series_df['Series ID'] == series]
        predView = series_df['Predicted View'].values[0]

        if save_only_desired:
            if predView in desired_series and series_df['Confidence'].values > confidence_value:
                for i, row in new.iterrows():
                    fileName = row['Modality'] + '.' + row['Series ID'] + '.' + row['Instance Number'] + '.dcm'
                    patientID = row['Patient ID'].upper()      
                    dicom = pydicom.dcmread(row['Filename'])

                    # save files to a 2-tier nested folder structure
                    if not os.path.exists(os.path.join(dst, patientID)):
                        os.makedirs(os.path.join(dst, patientID))

                    if not os.path.exists(os.path.join(dst, patientID, predView)):
                        os.makedirs(os.path.join(dst, patientID, predView))

                    dicom.save_as(os.path.join(dst, patientID, predView, fileName))
            else:
                pass
        else:
            for i, row in new.iterrows():
                fileName = row['Modality'] + '.' + row['Series ID'] + '.' + row['Instance Number'] + '.dcm'
                patientID = row['Patient ID'].upper()      
                dicom = pydicom.dcmread(row['Filename'])

                # save files to a 2-tier nested folder structure
                if not os.path.exists(os.path.join(dst, patientID)):
                    os.makedirs(os.path.join(dst, patientID))

                if not os.path.exists(os.path.join(dst, patientID, predView)):
                    os.makedirs(os.path.join(dst, patientID, predView))

                dicom.save_as(os.path.join(dst, patientID, predView, fileName))

Saving .csv file with series predictions and info


  0%|          | 0/22 [00:00<?, ?it/s]

Done!
Saving dicom files to new folder...


100%|██████████| 22/22 [00:11<00:00,  1.92it/s]


### Section 2.0 - Directory with Multiple Patients or Series

Use the following code to run the analysis over each patient and/or series in a directory individually. 

In [15]:
# PARAMETERS for the analysis
src = "../data/raw/dicoms/"                       # PATH to the directory containing the desired DICOM files (str)
dst = "../data/processed/sorted/"                 # PATH to the output directory to save dicom files (only valid if save_dicoms = True) (str)

modelname = 'ResNet50'                            # The neural network to load and used (Options: VGG19, ResNet50, or Xception)
modelpath = '../models/'                          # PATH to the saved models (str)

use_multiprocessing = True                        # Use multiprocessing to read header info (True or False)

# parameters for postprocessing/saving
csv_path = '../reports/series_predictions.csv'    # PATH to save the generated csv file (only valide if create_csv = True) (str)
create_csv = True                                 # Save a .csv file with the series level view predictions (True or False)
save_files = True                                 # Save dicom files to new directory (dst) (True or False)
save_only_desired = True                          # Save only dicom files corresponding to desired views (True or False)
confidence_value = 0.9                            # Only save series if the confidence is > a certain value (set to 0 to save all desired series, regardless of confidence) (float 0-1.0)

In [16]:
def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    
    return string.lower()  

def preprocess(img):
    # format image into tensor, standardized to 0-255
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(tf.expand_dims(img, 2), (224,224))
    img = tf.image.grayscale_to_rgb(img)
    
    # standardize
    img = img / np.max(img)
    img = img * 255.
    
    return img

def predict_view(img, model, classes):
    # make prediction on a single image
    pred = model.predict(tf.expand_dims(img, axis=0))
    pred = tf.argmax(pred, axis=-1)
    pred_view = classes[int(pred)]
    
    return pred_view

def batch_predict(batch, model, classes):
    # make prediction on a batch of images
    pred = model.predict(batch)
    pred = tf.argmax(pred, axis=-1)
    pred_view = [classes[int(x)] for x in pred]
    
    return pred_view

def get_dicom_header(dicom_loc):
    # read dicom file and return header information and image
    ds = pydicom.read_file(dicom_loc, force=True)
   
    # get patient, study, and series information
    patientID = clean_text(ds.get("PatientID", "NA"))
    studyDescription = clean_text(ds.get("StudyDescription", "NA"))
    seriesDescription = clean_text(ds.get("SeriesDescription", "NA"))
   
    # generate new, standardized file name
    modality = ds.get("Modality","NA")
    studyInstanceUID = ds.get("StudyInstanceUID","NA")
    seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
    seriesNumber = ds.get('SeriesNumber', 'NA')
    instanceNumber = str(ds.get("InstanceNumber","0"))
       
    # load image data
    array = ds.pixel_array
    
    return patientID, dicom_loc, modality, seriesInstanceUID, seriesNumber, instanceNumber, array, seriesDescription

In [17]:
def complete_view_prediction(directory, dst, modelname, modelpath,
                     csv_path,
                     create_csv,
                     use_multiprocessing,
                     save_files,
                     save_only_desired,
                     confidence_value):

    # Runs the complete view prediction over the dicom files in a directory
    # load the appropriate model
    #print('Loading model {}'.format(modelname))

    if modelname == 'ResNet50':
        MODELPATH = os.path.join(modelpath, 'Resnet/082621_resnet.hdf5')
        model = tf.keras.models.load_model(MODELPATH)
        #print(model.summary())

    elif modelname == 'VGG19':
        MODELPATH = os.path.join(modelpath, 'VGG19/vgg19.hdf5')
        model = tf.keras.models.load_model(MODELPATH)
        #print(model.summary())

    elif modelname == 'Xception':
        MODELPATH = os.path.join(modelpath, 'XCEPTION/xception.hdf5')
        model = tf.keras.models.load_model(MODELPATH)
        #print(model.summary())

    else:
        print('Uknown model specified in parameters!')

    # define possible class predictions
    classLabels = ['SA', '4CH', '2CH RT', 'RVOT', 'OTHER', '2CH LT', 'LVOT']
    classes = sorted(classLabels, key = str)

    # define the series that are needed/desired for cardiac modeling
    desired_series = ['4CH', 'SA', '2CH RT', '2CH LT', 'LVOT', 'RVOT']
    
    unsortedList = []
    for root, dirs, files in os.walk(directory):
        for file in files: 
            if ".dcm" in file: # exclude non-dicoms, good for messy folders
                unsortedList.append(os.path.join(root, file))

    #print('%s files found.' % len(unsortedList))

    if use_multiprocessing:
        with Pool(os.cpu_count()) as p:
            output = p.map(get_dicom_header, [dicom_loc for dicom_loc in unsortedList])

    else:
        output = []
        for dicom_loc in tqdm(unsortedList):
            output.append(get_dicom_header(dicom_loc))
            
    # generated pandas dataframe to store information from headers
    df = pd.DataFrame(sorted(output), columns = ['Patient ID', 
                                             'Filename',
                                             'Modality',
                                             'Series ID', 
                                             'Series Number', 
                                             'Instance Number', 
                                             'Img', 
                                             'Series Description'])
    
    output_series = []
    # make predictions and calculate confidence values
    for series in set(df['Series ID']):
        new = df[df['Series ID'] == series]

        dataset = tf.data.Dataset.from_tensor_slices([preprocess(x) for x in new['Img'].values])
        dataset = (dataset
                 .batch(16)
                 .prefetch(tf.data.experimental.AUTOTUNE))

        # record info for this series
        patient_id = new['Patient ID'].iloc[0]
        series_num = new['Series Number'].iloc[0]
        series_desc = new['Series Description'].iloc[0]
        frames = len(new)

        # make predictions over images
        views = batch_predict(dataset, model, classes)

        # find unique predictions and confidence for that series
        u, count = np.unique(views, return_counts=True)
        count_sort_ind = np.argsort(-count)
        pred = u[count_sort_ind][0]
        conf = np.round(np.max(count) / np.sum(count), 2)

        output_series.append([patient_id.upper(), series, series_num, frames, series_desc, pred, conf])

    output_series_df = pd.DataFrame(output_series, columns=['Patient ID', 'Series ID', 'Series Number', 'Frames', 'Series Description', 'Predicted View', 'Confidence'])
    
    if create_csv:
        #print('Saving .csv file with series predictions and info')
        if os.path.exists(csv_path):
            output_series_df.to_csv(csv_path, mode='a', header=False, index=False)
        else:
            output_series_df.to_csv(csv_path, mode='a', index=False)
        #print('Done!')

    if save_files:
        #print('Saving dicom files to new folder...')
        for series in output_series_df['Series ID']:
            new = df[df['Series ID'] == series]
            series_df = output_series_df[output_series_df['Series ID'] == series]
            predView = series_df['Predicted View'].values[0]
            patientID = series_df['Patient ID'].values[0].upper()

            if save_only_desired:
                if predView in desired_series and series_df['Confidence'].values > confidence_value:
                    for i, row in new.iterrows():
                        fileName = row['Modality'] + '.' + row['Series ID'] + '.' + row['Instance Number'] + '.dcm'   
                        dicom = pydicom.dcmread(row['Filename'])

                        # save files to a 2-tier nested folder structure
                        if not os.path.exists(os.path.join(dst, patientID)):
                            os.makedirs(os.path.join(dst, patientID))

                        if not os.path.exists(os.path.join(dst, patientID, predView)):
                            os.makedirs(os.path.join(dst, patientID, predView))

                        dicom.save_as(os.path.join(dst, patientID, predView, fileName))
                else:
                    pass
            else:
                for i, row in new.iterrows():
                    fileName = row['Modality'] + '.' + row['Series ID'] + '.' + row['Instance Number'] + '.dcm'
                    patientID = row['Patient ID'].upper()      
                    dicom = pydicom.dcmread(row['Filename'])

                    # save files to a 2-tier nested folder structure
                    if not os.path.exists(os.path.join(dst, patientID)):
                        os.makedirs(os.path.join(dst, patientID))

                    if not os.path.exists(os.path.join(dst, patientID, predView)):
                        os.makedirs(os.path.join(dst, patientID, predView))

                    dicom.save_as(os.path.join(dst, patientID, predView, fileName))

Run the complete view prediction for each series, in each subdirectory using the code below.

In [18]:
subdirectories = next(os.walk(src))[1]
print('Discovered {} subdirectories in source folder'.format(len(subdirectories)))

for subdir in tqdm(subdirectories):
    complete_view_prediction(os.path.join(src, subdir), dst=dst, modelname=modelname, modelpath=modelpath,  
                     csv_path=csv_path,
                     create_csv=create_csv,
                     use_multiprocessing=use_multiprocessing,
                     save_files=save_files,
                     save_only_desired=save_only_desired,
                     confidence_value=confidence_value)

  0%|          | 0/2 [00:00<?, ?it/s]

Discovered 2 subdirectories in source folder


100%|██████████| 2/2 [00:44<00:00, 22.46s/it]
